<a href="https://colab.research.google.com/github/phsieg/gan_artwork_open/blob/main/tf_gan_tutorial_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# README
This notebook is for testing the tutorial at https://www.tensorflow.org/tutorials/generative/cyclegan

# Find Local Modules Help

In [ ]:
help('modules')

# Testing Pad

In [ ]:
band = read_single_binary_file('./band.jpg')
band = read_multiple_binary_files('./test_folder/')


test_folder/IMG-20191121-WA0001.jpg is not a file. Skipping.
test_folder/Portrait_duo_artifact_triangles_2.jpg is not a file. Skipping.
test_folder/test_folder2 is not a file. Skipping.
test_folder/.ipynb_checkpoints is not a file. Skipping.


# File handling

In [ ]:
from pathlib import Path

def read_multiple_binary_files(path):
  p = Path(path)
  for child in p.iterdir():
    if child.is_dir:
      print(str(child) + " is not a file. Skipping.")
    else: 
      read_single_binary_file(child)

def read_single_binary_file(path):
  with open(path, 'rb') as f:
    return f.read()
  
def write_single_binary_file(path):
  with open(path, 'wb') as f:
    f.write()

# Tensorflow

In [ ]:
import tensorflow as tf
# for drawing generated images by plotting
import matplotlib.pyplot as plt 

# Program Execution


In [ ]:
isle = read_single_binary_file('.data/input/isle.png')
bandfoto = read_single_binary_file('.data/input/band.png')

# do ML stuff here ...

write_single_binary_file('.data/output/artwork.png')